# FVI Clustering

This notebook filters frames based on FVI scores, creates embeddings for the selected frames, and clusters the embeddings.

In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import random


# Add the path to the scripts directory
import sys
sys.path.append('../scripts')
from fvi_computation import compute_fvi

## Load FVI Scores

Load the FVI scores from the `fvi_scores.json` file.

In [ ]:
def load_fvi_scores(directory):
    fvi_scores = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file == 'fvi_scores.json':
                filepath = os.path.join(root, file)
                with open(filepath, 'r') as f:
                    scores = json.load(f)
                    fvi_scores[root] = scores
    return fvi_scores

fvi_scores = load_fvi_scores('../output/ground_truth')
fvi_scores

## Filter Frames

Filter frames with FVI scores above a certain threshold.

In [ ]:
threshold = 90  # Set your threshold here
selected_frames = []

for directory, scores in fvi_scores.items():
    for i, score in enumerate(scores):
        if score > threshold:
            frame_path = os.path.join(directory, f'{i+1:04d}.json')
            selected_frames.append(frame_path)

selected_frames

## Create Embeddings

Use a pre-trained ResNet model from PyTorch to create embeddings for the selected frames.

In [ ]:
model = models.densenet121(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # Remove the classification layer
model.eval()

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_embedding(img_path):
    img = Image.open(img_path).convert('RGB')
    img_tensor = preprocess(img)
    img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embedding = model(img_tensor)
    return embedding.squeeze().numpy()

embeddings = []

for frame_path in selected_frames:
    img_path = frame_path.replace('.json', '.jpg')  # Assuming images are in the same directory with .jpg extension I need the image files as well to create embeddings
    embedding = get_embedding(img_path)
    embeddings.append(embedding)

embeddings = np.array(embeddings)
embeddings.shape


## Cluster Embeddings

Apply KMeans clustering to the embeddings.

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 4  # Set the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

clusters

## Visualize Clusters

Use PCA to reduce the dimensionality of the embeddings and visualize the clusters.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=4)
reduced_embeddings = pca.fit_transform(embeddings)

plt.figure(figsize=(10, 8))
for i in range(n_clusters):
    cluster_points = reduced_embeddings[clusters == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {i}')
plt.legend()
plt.title('Clusters of Frames')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

# Visualize Clusters with FiftyOne

import fiftyone as fo
import fiftyone.zoo as foz

# Create a FiftyOne dataset for the clustered samples
clustered_dataset = fo.Dataset(name="clustered_dataset")

# Add samples with cluster labels to the new dataset
for sample in samples:
    clustered_sample = fo.Sample(filepath=sample.filepath, cluster=sample.cluster)
    clustered_dataset.add_sample(clustered_sample)

# Launch the FiftyOne app to visualize the clustered datase Python version 3.11 is not yet supported by FiftyOne. Please use a different version of Python. Used 3.10 to install
session = fo.launch_app(clustered_dataset)

# Display the clusters in the FiftyOne app
session.view = clustered_dataset.filter_labels("cluster", fo.ViewField("cluster").is_in([0, 1, 2]))